In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
# Load the dataset
raw_data = pd.read_csv('multimodal_tweets_dataset.csv')
raw_data.head()

,tweet_text,images,target,anotasi
0,"Kerajaan Pajajaran, Majapahit, Mataram Kuno &a...",1587087261275152384_0.jpg,muslim,not hate
1,Nih klub ngawur banget. Masa kampanye kaum pel...,1759555332689609118_0.jpg,lgbt,hate
2,Perbandingan Malaysia dan Indonesia dalam hal ...,1741940075737608656_0.jpg,rohingya,not hate
3,Di tunggu y KLARIFIKASINYA Bpk. Agus Subianto ...,1723960967657377808_0.jpg,muslim,hate
4,teruslah produktif dan terlihat menghambur2kan...,1776650672886460730_0.jpg,wanita,not hate


In [9]:
# drop images and target columns
data = raw_data.drop(columns=['images', 'target'])
data.head()

,tweet_text,anotasi
0,"Kerajaan Pajajaran, Majapahit, Mataram Kuno &a...",not hate
1,Nih klub ngawur banget. Masa kampanye kaum pel...,hate
2,Perbandingan Malaysia dan Indonesia dalam hal ...,not hate
3,Di tunggu y KLARIFIKASINYA Bpk. Agus Subianto ...,hate
4,teruslah produktif dan terlihat menghambur2kan...,not hate


In [10]:
# rename tweet_text to text and anotasi to label
data = data.rename(columns={'tweet_text': 'text', 'anotasi': 'label'})
data.head()

,text,label
0,"Kerajaan Pajajaran, Majapahit, Mataram Kuno &a...",not hate
1,Nih klub ngawur banget. Masa kampanye kaum pel...,hate
2,Perbandingan Malaysia dan Indonesia dalam hal ...,not hate
3,Di tunggu y KLARIFIKASINYA Bpk. Agus Subianto ...,hate
4,teruslah produktif dan terlihat menghambur2kan...,not hate


In [11]:
# Convert the value of text is string
data['text'] = data['text'].astype(str)

# convert not hate to neutral
data['label'] = data['label'].replace({'not hate': 'neutral'})

data.head(20)

,text,label
0,"Kerajaan Pajajaran, Majapahit, Mataram Kuno &a...",neutral
1,Nih klub ngawur banget. Masa kampanye kaum pel...,hate
2,Perbandingan Malaysia dan Indonesia dalam hal ...,neutral
3,Di tunggu y KLARIFIKASINYA Bpk. Agus Subianto ...,hate
4,teruslah produktif dan terlihat menghambur2kan...,neutral
5,"Alhamdulillah taun ini masih bisa berbagi, eve...",neutral
6,"Assalamualaikum, yg ga jwb kris10 https://t.co...",neutral
7,Gini ya Sobat Gurun . Menyan itu digunakan jug...,hate
8,Nabi lahir di Mekah meninggal di Medinah. Di k...,hate
9,USER Jangan bawa2 cina sipit Bu... Kan golonga...,hate


In [14]:
# Atau jika ingin lebih spesifik (hashtag dengan huruf/angka/underscore)
data['text'] = data['text'].str.replace(r'#[A-Za-z0-9_]+', '', regex=True)

# Hapus mention (username Twitter)
data['text'] = data['text'].str.replace(r'@\w+', '', regex=True)

# Hapus hashtag
data['text'] = data['text'].str.replace(r'#\w+', '', regex=True)

# Hapus angka di depan text
data['text'] = data['text'].str.replace(r'^\d+\.?\s*', '', regex=True)

In [15]:
import re

def clean_text(text):
    # Hapus URL/link
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'https://t\.co/\w+', '', text)
    
    # Hapus RT di awal
    text = re.sub(r'^RT\s*:?\s*', '', text, flags=re.IGNORECASE)
    
    text = re.sub(r'^dips\s*:?\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'^tanyarl\s*:?\s*', '', text, flags=re.IGNORECASE)
    
    # Hapus USER (case insensitive)
    text = re.sub(r'\bUSER\b', '', text, flags=re.IGNORECASE)
    
    # Hapus tanda - di depan
    text = re.sub(r'^-\s*', '', text)
    
    # Hapus emoji atau karakter khusus
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Hapus whitespace berlebih
    text = re.sub(r'\s+', ' ', text)
    
    # Hapus spasi di awal dan akhir
    return text.strip()

# Apply cleaning
data['text'] = data['text'].apply(clean_text)

data.head(20)

,text,label
0,Kerajaan Pajajaran Majapahit Mataram Kuno amp ...,neutral
1,Nih klub ngawur banget Masa kampanye kaum pela...,hate
2,Perbandingan Malaysia dan Indonesia dalam hal ...,neutral
3,Di tunggu y KLARIFIKASINYA Bpk Agus Subianto u...,hate
4,teruslah produktif dan terlihat menghambur2kan...,neutral
5,Alhamdulillah taun ini masih bisa berbagi even...,neutral
6,Assalamualaikum yg ga jwb kris10,neutral
7,Gini ya Sobat Gurun Menyan itu digunakan juga ...,hate
8,Nabi lahir di Mekah meninggal di Medinah Di ke...,hate
9,Jangan bawa2 cina sipit Bu Kan golonganmu jg c...,hate


In [16]:
data.head(50)

,text,label
0,Kerajaan Pajajaran Majapahit Mataram Kuno amp ...,neutral
1,Nih klub ngawur banget Masa kampanye kaum pela...,hate
2,Perbandingan Malaysia dan Indonesia dalam hal ...,neutral
3,Di tunggu y KLARIFIKASINYA Bpk Agus Subianto u...,hate
4,teruslah produktif dan terlihat menghambur2kan...,neutral
5,Alhamdulillah taun ini masih bisa berbagi even...,neutral
6,Assalamualaikum yg ga jwb kris10,neutral
7,Gini ya Sobat Gurun Menyan itu digunakan juga ...,hate
8,Nabi lahir di Mekah meninggal di Medinah Di ke...,hate
9,Jangan bawa2 cina sipit Bu Kan golonganmu jg c...,hate


In [17]:
# save to csv
data.to_csv('cleaned_1.csv', index=False)